<a href="https://colab.research.google.com/github/jashvidesai/ORF-Thesis/blob/main/ALNSInitialSolnUpdated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import heapq
import random

# Problem Parameters
V = range(7)
V_star = range(1, 7)
K = range(4)
Q = [25, 18, 20, 23]
vehicle_speed = 60
fixed_costs = [110, 100, 105, 108]

# distance matrix
np.random.seed(42)
distances = np.array([
    [0, 35, 18, 23, 27, 48, 19],
    [35, 0, 25, 37, 34, 38, 31],
    [18, 25, 0, 27, 25, 30, 24],
    [23, 37, 27, 0, 40, 35, 26],
    [27, 34, 25, 40, 0, 26, 17],
    [48, 38, 30, 35, 26, 0, 39],
    [19, 31, 24, 26, 17, 39, 0]
])

# travel times and costs
t = distances / vehicle_speed
c = distances * 0.093

# delivery and pickup Demands
d = [0, 50, 35, 20, 25, 30, 25]
p = [0, 30, 22.5, 30, 45, 40, 28]

# time windows and service times
a = [0, 5, 10, 20, 20, 5, 10]  # earliest arrival times
b = [100, 100, 100, 100, 100, 100, 100]  # latest departure times
s = [0, 10, 10, 7, 5, 5, 10]  # service times
'''
V = range(15)
V_star = range(1, 15)
K = range(4)
Q = [40, 35, 30, 25]

np.random.seed(42)
distances = np.array([
    [  0,  12,  22,  30,  25,  18,  20,  45,  28,  32,  27,  40,  35,  50,  38],
    [ 12,   0,  15,  27,  22,  20,  17,  40,  30,  25,  20,  37,  32,  45,  40],
    [ 22,  15,   0,  18,  25,  22,  19,  35,  40,  38,  30,  45,  37,  50,  42],
    [ 30,  27,  18,   0,  22,  20,  18,  32,  45,  37,  35,  42,  38,  50,  47],
    [ 25,  22,  25,  22,   0,  18,  20,  30,  35,  28,  25,  40,  32,  47,  38],
    [ 18,  20,  22,  20,  18,   0,  12,  27,  30,  22,  18,  35,  28,  42,  35],
    [ 20,  17,  19,  18,  20,  12,   0,  25,  32,  20,  22,  38,  27,  45,  38],
    [ 45,  40,  35,  32,  30,  27,  25,   0,  40,  37,  35,  42,  38,  50,  47],
    [ 28,  30,  40,  45,  35,  30,  32,  40,   0,  15,  20,  28,  25,  35,  30],
    [ 32,  25,  38,  37,  28,  22,  20,  37,  15,   0,  12,  30,  20,  42,  35],
    [ 27,  20,  30,  35,  25,  18,  22,  35,  20,  12,   0,  25,  15,  37,  28],
    [ 40,  37,  45,  42,  40,  35,  38,  42,  28,  30,  25,   0,  18,  28,  25],
    [ 35,  32,  37,  38,  32,  28,  27,  38,  25,  20,  15,  18,   0,  22,  18],
    [ 50,  45,  50,  50,  47,  42,  45,  50,  35,  42,  37,  28,  22,   0,  12],
    [ 38,  40,  42,  47,  38,  35,  38,  47,  30,  35,  28,  25,  18,  12,   0]
])

vehicle_speed = 60  # vehicle speed in km/h
t = distances / vehicle_speed  # travel times
c = distances * 0.093  # travel costs (arbitrary scaling for fuel)
fixed_costs = [120, 115, 110, 100]

d = [0, 15, 10, 12, 8,  14, 9,  11, 10, 7,  12, 14, 13, 6,  9]
p = [0,  5,  3,  6, 4,   7, 2,   4,  6, 3,   5,  8,  7, 2,  3]
a = [0,  5,  10,  8,  12,  15,  18,  20,  10,  15,  18,  10,  12,  15,  18]
b = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
s = [0,  5,  5,  6, 4,  5,  6,  5,  6, 4,  5,  6,  7, 3,  5]
'''
# Vehicle Class
class Vehicle:
    def __init__(self, vehicle_id, capacity, speed, cost, depot):
        self.vehicle_id = vehicle_id
        self.capacity = capacity
        self.remaining_capacity = 0
        self.speed = speed
        self.cost = cost
        self.route = []  # Stores (customer_id, delivered, picked_up, arrival_time)
        self.current_time = 0
        #self.current_load = capacity # start full
        self.current_load = int(self.capacity * 0.5)  # Start with 50-90% of capacity
        self.current_location = depot

    def can_add_customer(self, node, travel_time):
        """
        Check if a vehicle can arrive at the node within the allowed time window.
        Returns a boolean feasibility flag and the expected arrival time.
        """
        arrival_time = self.current_time + travel_time
        if arrival_time > b[node]:  # Arrives too late
            return False, arrival_time
        if arrival_time < a[node]:  # Arrives too early, wait
            arrival_time = a[node]
        return True, arrival_time

    def add_customer(self, node, delivery, pickup, travel_time):
        """
        Assigns a customer to the vehicle while ensuring delivery before pickup
        and allowing split deliveries.
        """
        feasible, arrival_time = self.can_add_customer(node, travel_time)
        if not feasible:
            return False, delivery, pickup

        # Step 1: Drop Off First
        delivered = min(self.current_load, delivery)
        self.current_load -= delivered  # Reduce load after delivery
        self.remaining_capacity = self.capacity - self.current_load
        delivery -= delivered

        # Step 2: Pick Up
        picked_up = min(self.remaining_capacity, pickup)
        self.current_load += picked_up
        self.remaining_capacity -= picked_up
        pickup -= picked_up

        # Step 3: Save Route & Update Time
        self.route.append((node, delivered, picked_up, arrival_time))
        self.current_time = arrival_time + s[node]
        self.current_location = node

        print(f"Vehicle {self.vehicle_id} assigned customer {node}")
        return True, delivery, pickup

In [5]:
def compute_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the farthest customer from the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    # finds the farthest customer from the depot
    start_node = max(V_star, key=lambda c: distances[depot][c])
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        # find the closest node to the last added node
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

def generate_sequential_solution(V, V_star, K, Q, d, p, a, b, s, t, depot, distances):
    """
    Deploys vehicles sequentially, serving customers in the order computed above
    while ensuring split deliveries.
    """
    # Precompute the visiting order
    ordered_route = compute_ordered_route(V_star, depot, distances)

    # Sort vehicles by descending capacity
    vehicles = sorted([Vehicle(k, Q[k], vehicle_speed, fixed_costs[k], depot) for k in K], key=lambda v: v.capacity, reverse=True)

    remaining_deliveries = {i: d[i] for i in V_star}  # Track remaining deliveries
    remaining_pickups = {i: p[i] for i in V_star}  # Track remaining pickups

    for vehicle in vehicles:
        print(f"\n Deploying Vehicle {vehicle.vehicle_id} (Capacity {vehicle.capacity})\n")

        for customer in ordered_route[:]:
            if remaining_deliveries[customer] <= 0 and remaining_pickups[customer] <= 0:
                # Skip nodes that no longer need service
                print(f" Skipping Customer {customer}, fully served.")
                ordered_route.remove(customer)
                continue

            print(f"\n Vehicle {vehicle.vehicle_id} starting route. Current Load: {vehicle.current_load}, Remaining Capacity: {vehicle.remaining_capacity}")
            print(f" Attempting to serve Customer {customer}")

            # get demand values to print
            prev_delivery = remaining_deliveries[customer]
            prev_pickup = remaining_pickups[customer]

            print(f"Before Service: Delivery Needed = {prev_delivery}, Pickup Needed = {prev_pickup}")

            # serve the customer
            success, new_remaining_delivery, new_remaining_pickup = vehicle.add_customer(
                customer, prev_delivery, prev_pickup, t[vehicle.current_location][customer]
            )

            # if the vehicle cannot serve, it returns to the depot
            if not success:
                print(f"Vehicle {vehicle.vehicle_id} cannot serve Customer {customer}, returning to depot.")
                break

            # update remaining demand
            remaining_deliveries[customer] = max(0, new_remaining_delivery)
            remaining_pickups[customer] = max(0, new_remaining_pickup)

            # Update vehicle state
            vehicle.current_location = customer
            delivered = min(prev_delivery, vehicle.current_load)
            vehicle.current_load -= delivered
            picked_up = min(prev_pickup, vehicle.capacity - vehicle.current_load)
            vehicle.current_load += picked_up
            vehicle.remaining_capacity = vehicle.capacity - vehicle.current_load

            print(f"After Service: Delivery Left = {remaining_deliveries[customer]}, Pickup Left = {remaining_pickups[customer]}")
            print(f"Vehicle {vehicle.vehicle_id} served Customer {customer}")
            print(f"Vehicle {vehicle.vehicle_id} Updated Load: {vehicle.current_load}, Remaining Capacity: {vehicle.remaining_capacity}")

            # if the node is fully served, remove it
            if remaining_deliveries[customer] <= 0 and remaining_pickups[customer] <= 0:
                print(f"Customer {customer} fully served. Removing from route.")
                ordered_route.remove(customer)

        # return to depot when done
        print(f"Vehicle {vehicle.vehicle_id} returning to depot.")
        depot_travel_time = t[vehicle.current_location][depot]
        vehicle.current_time += depot_travel_time
        vehicle.route.append((depot, 0, 0, vehicle.current_time))

    return vehicles


# Run the new sequential approach
vehicles = generate_sequential_solution(V, V_star, K, Q, d, p, a, b, s, t, depot=0, distances=distances)

# Print final routes
print("\n **Final Vehicle Routes** ")
for v in vehicles:
    print(f"Vehicle {v.vehicle_id}: Route {v.route}, Final Load: {v.current_load}")


 Deploying Vehicle 0 (Capacity 25)


 Vehicle 0 starting route. Current Load: 12, Remaining Capacity: 0
 Attempting to serve Customer 5
Before Service: Delivery Needed = 30, Pickup Needed = 40
Vehicle 0 assigned customer 5
After Service: Delivery Left = 18, Pickup Left = 15
Vehicle 0 served Customer 5
Vehicle 0 Updated Load: 25, Remaining Capacity: 0

 Vehicle 0 starting route. Current Load: 25, Remaining Capacity: 0
 Attempting to serve Customer 4
Before Service: Delivery Needed = 25, Pickup Needed = 45
Vehicle 0 assigned customer 4
After Service: Delivery Left = 0, Pickup Left = 20
Vehicle 0 served Customer 4
Vehicle 0 Updated Load: 25, Remaining Capacity: 0

 Vehicle 0 starting route. Current Load: 25, Remaining Capacity: 0
 Attempting to serve Customer 6
Before Service: Delivery Needed = 25, Pickup Needed = 28
Vehicle 0 assigned customer 6
After Service: Delivery Left = 0, Pickup Left = 3
Vehicle 0 served Customer 6
Vehicle 0 Updated Load: 25, Remaining Capacity: 0

 Vehicle 0 sta